In [1]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat
from rsome import ro
from rsome import grb_solver as grb

# Para b_media

# Intervalo entorno a la media

Importar datos del problema

In [2]:
data = loadmat("Datos_Manufacturing_media.mat")
A_media = data['A_media']
A_hat_media = data['A_hat_media']
b_media = data['b_media'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0])

In [3]:
# === Parámetros del problema ===
m, n = A_media.shape

# Todos los coeficientes son inciertos
J = [list(range(n)) for _ in range(m)]

# === Probar con distintos valores de Gamma ===
for Gamma_value in [1.0, 2.0, 3.0, 4.0, 5.0]:
    print(f"\n--- Resolviendo con Γ = {Gamma_value} ---")

    Gamma = np.array([Gamma_value] * m)
    model = ro.Model()

    x = model.dvar(n)
    y = model.dvar(n)
    z = model.dvar(m)
    p = {}

    # Declarar variables auxiliares p_ij
    for i in range(m):
        for j in J[i]:
            p[i, j] = model.dvar()

    # Restricciones robustas
    for i in range(m):
        expr = sum(A_media[i, j] * x[j] for j in range(n)) + Gamma[i] * z[i] + sum(p[i, j] for j in J[i])
        model.st(expr <= b_media[i])

        for j in J[i]:
            model.st(z[i] + p[i, j] >= A_hat_media[i, j] * y[j])
            model.st(p[i, j] >= 0)

    # Relación y_j = |x_j|
    for j in range(n):
        model.st(x[j] <= y[j])
        model.st(-x[j] <= y[j])
        model.st(y[j] >= 0)

    # Dominio de x
    model.st(x >= 0)

    # Objetivo
    model.max(c @ x)

    # Resolver
    model.solve(grb)

    # Resultados
    x_val = x.get()
    obj_val = model.get()
    print(f"Solución x* = {np.round(x_val, 4)}")
    print(f"Valor objetivo = {round(obj_val, 4)}")


--- Resolviendo con Γ = 1.0 ---
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2679891
Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [3.1136 4.6704 0.     5.1894 9.3409]
Valor objetivo = 22.3143

--- Resolviendo con Γ = 2.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [2.3428 3.5142 5.0059 3.9047 7.0285]
Valor objetivo = 21.7962

--- Resolviendo con Γ = 3.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [ 0.      0.      0.      0.     21.6131]
Valor objetivo = 21.6131

--- Resolviendo con Γ = 4.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0060s
Solución x* = [ 0.      0.      0

# Intervalo [min max]

In [4]:
data = loadmat("Datos_Manufacturing_MinMax.mat")
A_MinMax = data['A_MinMax']
A_hat_MinMax = data['A_hat_MinMax']
b_media = data['b_media'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [5]:
# === Parámetros del problema ===
m, n = A_MinMax.shape

# Todos los coeficientes son inciertos
J = [list(range(n)) for _ in range(m)]

# === Probar con distintos valores de Gamma ===
for Gamma_value in [1.0, 2.0, 3.0, 4.0, 5.0]:
    print(f"\n--- Resolviendo con Γ = {Gamma_value} ---")

    Gamma = np.array([Gamma_value] * m)
    model = ro.Model()

    x = model.dvar(n)
    y = model.dvar(n)
    z = model.dvar(m)
    p = {}

    # Declarar variables auxiliares p_ij
    for i in range(m):
        for j in J[i]:
            p[i, j] = model.dvar()

    # Restricciones robustas
    for i in range(m):
        expr = sum(A_MinMax[i, j] * x[j] for j in range(n)) + Gamma[i] * z[i] + sum(p[i, j] for j in J[i])
        model.st(expr <= b_media[i])

        for j in J[i]:
            model.st(z[i] + p[i, j] >= A_hat_MinMax[i, j] * y[j])
            model.st(p[i, j] >= 0)

    # Relación y_j = |x_j|
    for j in range(n):
        model.st(x[j] <= y[j])
        model.st(-x[j] <= y[j])
        model.st(y[j] >= 0)

    # Dominio de x
    model.st(x >= 0)

    # Objetivo
    model.max(c @ x)

    # Resolver
    model.solve(grb)

    # Resultados
    x_val = x.get()
    obj_val = model.get()
    print(f"Solución x* = {np.round(x_val, 4)}")
    print(f"Valor objetivo = {round(obj_val, 4)}")


--- Resolviendo con Γ = 1.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [4.0413 4.0821 4.0413 4.0012 4.0821]
Valor objetivo = 20.2479

--- Resolviendo con Γ = 2.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [3.5944 3.6307 3.5944 3.5589 3.6307]
Valor objetivo = 18.0092

--- Resolviendo con Γ = 3.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [3.2366 3.2693 3.2366 3.2045 3.2693]
Valor objetivo = 16.2163

--- Resolviendo con Γ = 4.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [2.9435 2.9733 2.9435 2.9144 2.9733]
Valor objetivo = 14.748

--- Resolviendo con Γ = 5.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solució

# Para b_max

## Intervalo entorno a la media

Importar datos del problema

In [6]:
b_max = data['b_max'].flatten()

In [7]:
# === Parámetros del problema ===
m, n = A_media.shape

# Todos los coeficientes son inciertos
J = [list(range(n)) for _ in range(m)]

# === Probar con distintos valores de Gamma ===
for Gamma_value in [1.0, 2.0, 3.0, 4.0, 5.0]:
    print(f"\n--- Resolviendo con Γ = {Gamma_value} ---")

    Gamma = np.array([Gamma_value] * m)
    model = ro.Model()

    x = model.dvar(n)
    y = model.dvar(n)
    z = model.dvar(m)
    p = {}

    # Declarar variables auxiliares p_ij
    for i in range(m):
        for j in J[i]:
            p[i, j] = model.dvar()

    # Restricciones robustas
    for i in range(m):
        expr = sum(A_media[i, j] * x[j] for j in range(n)) + Gamma[i] * z[i] + sum(p[i, j] for j in J[i])
        model.st(expr <= b_max[i])

        for j in J[i]:
            model.st(z[i] + p[i, j] >= A_hat_media[i, j] * y[j])
            model.st(p[i, j] >= 0)

    # Relación y_j = |x_j|
    for j in range(n):
        model.st(x[j] <= y[j])
        model.st(-x[j] <= y[j])
        model.st(y[j] >= 0)

    # Dominio de x
    model.st(x >= 0)

    # Objetivo
    model.max(c @ x)

    # Resolver
    model.solve(grb)

    # Resultados
    x_val = x.get()
    obj_val = model.get()
    print(f"Solución x* = {np.round(x_val, 4)}")
    print(f"Valor objetivo = {round(obj_val, 4)}")


--- Resolviendo con Γ = 1.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...


Solution status: 2
Running time: 0.0020s
Solución x* = [ 4.2549  6.3824  0.      6.6793 12.7648]
Valor objetivo = 30.0816

--- Resolviendo con Γ = 2.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [ 3.404   5.1061  3.7959  5.6734 11.3468]
Valor objetivo = 29.3262

--- Resolviendo con Γ = 3.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [ 0.      0.      0.      0.     29.1455]
Valor objetivo = 29.1455

--- Resolviendo con Γ = 4.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [ 0.      0.      0.      0.     29.1455]
Valor objetivo = 29.1455

--- Resolviendo con Γ = 5.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [ 0.      0.      0.      0.     29.1455]
Valor objetivo = 29.1455


# Intervalo [min max]

In [8]:
b_max = data['b_max'].flatten()

In [9]:
# === Parámetros del problema ===
m, n = A_MinMax.shape

# Todos los coeficientes son inciertos
J = [list(range(n)) for _ in range(m)]

# === Probar con distintos valores de Gamma ===
for Gamma_value in [1.0, 2.0, 3.0, 4.0, 5.0]:
    print(f"\n--- Resolviendo con Γ = {Gamma_value} ---")

    Gamma = np.array([Gamma_value] * m)
    model = ro.Model()

    x = model.dvar(n)
    y = model.dvar(n)
    z = model.dvar(m)
    p = {}

    # Declarar variables auxiliares p_ij
    for i in range(m):
        for j in J[i]:
            p[i, j] = model.dvar()

    # Restricciones robustas
    for i in range(m):
        expr = sum(A_MinMax[i, j] * x[j] for j in range(n)) + Gamma[i] * z[i] + sum(p[i, j] for j in J[i])
        model.st(expr <= b_max[i])

        for j in J[i]:
            model.st(z[i] + p[i, j] >= A_hat_MinMax[i, j] * y[j])
            model.st(p[i, j] >= 0)

    # Relación y_j = |x_j|
    for j in range(n):
        model.st(x[j] <= y[j])
        model.st(-x[j] <= y[j])
        model.st(y[j] >= 0)

    # Dominio de x
    model.st(x >= 0)

    # Objetivo
    model.max(c @ x)

    # Resolver
    model.solve(grb)

    # Resultados
    x_val = x.get()
    obj_val = model.get()
    print(f"Solución x* = {np.round(x_val, 4)}")
    print(f"Valor objetivo = {round(obj_val, 4)}")


--- Resolviendo con Γ = 1.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [5.2965 5.313  5.3798 5.342  5.5185]
Valor objetivo = 26.8498

--- Resolviendo con Γ = 2.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [4.6835 4.698  4.757  4.7236 4.8797]
Valor objetivo = 23.7418

--- Resolviendo con Γ = 3.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Solución x* = [4.1976 4.2106 4.2635 4.2336 4.3735]
Valor objetivo = 21.2788

--- Resolviendo con Γ = 4.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [3.803  3.8148 3.8628 3.8357 3.9624]
Valor objetivo = 19.2787

--- Resolviendo con Γ = 5.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s
Soluci